In [156]:
MAX_SEQ_LEN = 15


def GlobalParams():
    global MAX_SEQ_LEN

In [243]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [273]:
# ① 데이터 읽어오기
train_file = os.path.join('./models/intent', "test_sum.csv")
data = pd.read_csv(train_file, delimiter=',')
data.columns = ['intent','query']
data

,intent,query
0,0,"음, 그러면 하나 해주세요."
1,1,네. 자동으로 하면 디씨 되나요?
2,2,어떤 게 있나요?
3,2,네. 내추럴 브라운이랑 두 가지 안 되나요?
4,2,어떤 색깔을 주문해야 되나요?
...,...,...
127847,10,뷰티
127848,10,아우터
127849,10,스커트
127850,10,뷰티


In [274]:
data.iloc[0]['query'].find('추천')

-1

In [275]:
for i in data:
    if data.iloc[0]['query'].find('추천') >0:
        data.iloc[0]['intent'] = 8

In [276]:
data['intent'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
      dtype=int64)

In [277]:
data[data['intent'] == 12]  # 상품색상문의에 대한 문장

,intent,query
8404,12,아니오
8405,12,아니에요
8406,12,아니
8407,12,아냐
8408,12,아니요
...,...,...
117943,12,nO
117944,12,No
117945,12,하지마
117946,12,이러지마 제발


In [278]:
queries = data.dropna()['query'].tolist()
intents = data.dropna()['intent'].tolist()

In [279]:
len(queries)

127852

In [280]:
data.dropna()['intent'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
      dtype=int64)

In [281]:
queries[:20]

['음, 그러면 하나 해주세요.',
 '네. 자동으로 하면 디씨 되나요?',
 '어떤 게 있나요?',
 '네. 내추럴 브라운이랑 두 가지 안 되나요?',
 '어떤 색깔을 주문해야 되나요?',
 '아. 다크가 좀 진한 색이죠?',
 '제가 프라이팬으로 착각했지만 이 상품 박스채 그대로라 반품해주세요.',
 '그거 같은 가격에 살 수 있어요?',
 '네, 주문은 맞는데 취소할려고요.',
 '내가 삼십육이니까.',
 '네. 사만 팔천구백원 아닙니까?',
 '바닥에 하는 거 신청하려구요.',
 '이백삼십오말고 이백사십으로 되나요?',
 '예. 타일 박사 신청하려고요.',
 '네 그런데 칠만 사천 백원으로 뜨는 건 뭐예요?',
 '그러면 취소 좀 시켜주세요.',
 '그러니까 배려를 좀 해주세요.',
 '네. 확인이 안 되면 취소해 주세요.',
 '천원 할인이 적용되는 거죠?',
 '육만구천구백원이요?']

In [282]:
intents[:20]

[0, 1, 2, 2, 2, 2, 3, 4, 5, 6, 4, 0, 6, 0, 4, 5, 5, 5, 1, 4]

In [283]:
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [284]:
queries[30829]

'마카로니 도와줘'

In [285]:
sequences = []
cnt=0
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)
    cnt+=1

In [286]:
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [287]:
# ③ 학습용, 검증용, 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries)) # 랜덤 섞기

# 학습셋:검증셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.65)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.15)

train_ds = ds.take(train_size).batch(11)
val_ds = ds.skip(train_size).take(val_size).batch(11)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(11)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

In [288]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입력크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(14, name='logits')(dropout_hidden)  # 최종적으로 14개의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(14, activation=tf.nn.softmax)(logits)

In [289]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [290]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 15)]         0           []                               
                                                                                                  
 embedding_11 (Embedding)       (None, 15, 128)      374656      ['input_12[0][0]']               
                                                                                                  
 dropout_22 (Dropout)           (None, 15, 128)      0           ['embedding_11[0][0]']           
                                                                                                  
 conv1d_33 (Conv1D)             (None, 13, 128)      49280       ['dropout_22[0][0]']             
                                                                                           

In [291]:
# # 모델 학습
# # ★ 시간 걸림 ★

model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
7555/7555 [==============================] - 52s 7ms/step - loss: 0.2050 - accuracy: 0.9283 - val_loss: 0.1148 - val_accuracy: 0.9519
Epoch 2/5
7555/7555 [==============================] - 53s 7ms/step - loss: 0.1328 - accuracy: 0.9485 - val_loss: 0.0969 - val_accuracy: 0.9560
Epoch 3/5
7555/7555 [==============================] - 52s 7ms/step - loss: 0.1204 - accuracy: 0.9515 - val_loss: 0.0910 - val_accuracy: 0.9585
Epoch 4/5
7555/7555 [==============================] - 53s 7ms/step - loss: 0.1141 - accuracy: 0.9533 - val_loss: 0.0907 - val_accuracy: 0.9553
Epoch 5/5
7555/7555 [==============================] - 54s 7ms/step - loss: 0.1106 - accuracy: 0.9546 - val_loss: 0.0924 - val_accuracy: 0.9587


In [292]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

1744/1744 [==============================] - 5s 3ms/step - loss: 0.0913 - accuracy: 0.9583
Accuracy: 95.833552
loss: 0.091349


In [293]:
# ⑧ 모델 저장
model.save(os.path.join('./models/intent','intent_model.h5'))

In [294]:
from utils.Preprocess import Preprocess    # 의도분류 모델 테스트
from models.intent.IntentModel import IntentModel
import os

p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [295]:
intent = IntentModel(model_name=os.path.join('./models/intent', 'intent_model.h5'), preprocess=p)

In [300]:
query = '아우터'
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

아우터
의도 예측 클래스 :  10
의도 예측 레이블 :  카테고리선택


In [297]:
p.pos('동의하지')

[('동의', 'NNG'), ('하', 'XSV'), ('지', 'EC')]